In [1]:
import os

os.chdir(r'C:\Users\carlo\GitHub\ProjectBuffalo')

In [2]:
import buffalo.ingestion as ingestion

In [3]:
ingestion.configuration.Configuration.print()

{'storage_folder': '../__cached_data', 'api_keys': {<API.ADVANTAGE: 1>: '2rrNROO0beX90lPH7ixQOp0mT_9SwF0d'}, 'watched_tickers': ['IBM', 'MSFT']}


In [4]:
advantage_endpoint = ingestion.advantage.AdvantageStockGrepper()

In [5]:
apple_stock = advantage_endpoint.stock_download(
    symbol='AAPL',
    interval='daily',
    year_slice=None, ## Intraday 
    adjusted=True
)

In [9]:
advantage_endpoint.company_info_download(
    'CASH_FLOW',
    'IBM',
    '3month'
)

,symbol,annualReports,quarterlyReports
0,IBM,"[{'fiscalDateEnding': '2022-12-31', 'reportedC...","[{'fiscalDateEnding': '2022-12-31', 'reportedC..."


In [10]:
advantage_endpoint.listing_info_download(date='2022-02-24')

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,NaN,Active
...,...,...,...,...,...,...,...
12039,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active
12040,ZYRX,Global Earnings Capital Ltd,NASDAQ,Stock,2007-07-13,NaN,Active
12041,ZYXI,Zynex Inc,NASDAQ,Stock,2012-07-23,NaN,Active
12042,ZZK,NaN,NYSE ARCA,Stock,2020-07-22,NaN,Active


In [11]:
advantage_endpoint.trend_indicator_download(
    function='SMA',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,time,SMA
0,2023-03-15 19:58,123.3510
1,2023-03-15 19:53,123.3210
2,2023-03-15 19:38,123.3190
3,2023-03-15 19:30,123.2990
4,2023-03-15 18:38,123.2790
...,...,...
4096,2023-03-02 09:37,128.0535
4097,2023-03-02 09:36,128.0617
4098,2023-03-02 09:35,128.0802
4099,2023-03-02 09:34,128.0542


In [12]:
advantage_endpoint.trend_indicator_download(
    function='T3',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,time,T3
0,2023-03-15 19:58,123.3875
1,2023-03-15 19:53,123.3757
2,2023-03-15 19:38,123.3594
3,2023-03-15 19:30,123.3404
4,2023-03-15 18:38,123.3232
...,...,...
4051,2023-03-02 10:22,128.2305
4052,2023-03-02 10:21,128.2450
4053,2023-03-02 10:20,128.2476
4054,2023-03-02 10:19,128.2431


In [13]:
advantage_endpoint.econ_download(
    function='FEDERAL_FUNDS_RATE',
    interval='daily'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [14]:
advantage_endpoint.econ_download(
    function='TREASURY_YIELD',
    interval='daily',
    maturity='3month'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [15]:
advantage_endpoint.econ_download(
    function='NONFARM_PAYROLL',
    interval='monthly'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [16]:
advantage_endpoint.econ_download(
    function='CPI',
    interval='monthly'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [17]:
advantage_endpoint.econ_download(
    function='UNEMPLOYMENT',
    interval='monthly'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [18]:
advantage_endpoint.cycle_indicator_download(
    function='ULTOSC',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [19]:
advantage_endpoint.cycle_indicator_download(
    function='ULTOSC',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,{
0,"""Note"": ""Thank you for using Alpha Vantage..."
1,}


In [20]:
advantage_endpoint.market_news_sentiment_download('IBM', sort='Latest', limit=50)[0]

KeyError: "Key 'feed' not found. If specifying a record_path, all elements of data should have the path."

In [ ]:
advantage_endpoint.market_news_sentiment_download('IBM', sort='relevance', limit=50)[1]

In [ ]:
# import sqlite3


# fp = r'~/newsqlitefile.sqlite'

# import pandas as pd

# a = pd.DataFrame({"a": range(10), "v": range(1, 11)})

# newconn, _ = sqlite3.connect(fp)
# a.to_sql("a", newconn)

# newconn.close()